In [0]:
import os, warnings
warnings.filterwarnings(action='ignore')

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
total_epoch = 100 # epoch 수 설정
batch_size = 100 # 배치 사이즈
learning_rate = 0.0002 # 학습률
# 신경망 레이어 구성 옵션
n_hidden = 256 # 은닉층 노드
n_input = 28 * 28 # 입력
n_noise = 128 # 생성기의 입력값으로 사용할 노이즈의 크기

In [0]:
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈 Z를 입력값으로 사용합니다.
Z = tf.placeholder(tf.float32, [None, n_noise])

In [0]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [0]:
# 판별기 신경망에 사용하는 변수들입니다.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
# 판별기의 최종 결과값은 얼마나 진짜와 가깝냐를 판단하는 한 개의 스칼라값입니다.
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [0]:
def generator(noise_z):
    hidden = tf.nn.relu( tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid( tf.matmul(hidden, G_W2) + G_b2)
    
    return output

In [0]:
def discriminator(inputs):
    hidden = tf.nn.relu( tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid( tf.matmul(hidden, D_W2) + D_b2)
    return output

In [0]:
G = generator(Z)
# 노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구합니다.
D_fake = discriminator(G)
# 진짜 이미지를 이용해 판별한 값을 구합니다.
D_real = discriminator(X)

In [0]:
loss_D = tf.abs(tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_fake)))
loss_G = tf.abs(tf.reduce_mean(tf.log(D_fake)))

In [0]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [0]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G, var_list=G_var_list)

In [0]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [0]:
%%time
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        # 판별기와 생성기 신경망을 각각 학습시킵니다.
        _, loss_val_D = sess.run([train_D, loss_D],
        feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
        feed_dict={Z: noise})
    print('Epoch:', '%04d' % epoch,
        'D loss: {:.4}'.format(loss_val_D),
        'G loss: {:.4}'.format(loss_val_G))

    #########
    # 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
    ######
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise) # 노이즈 생성.
        samples = sess.run(G, feed_dict={Z: noise})
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        plt.savefig('{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
print('최적화 완료!')

Epoch: 0000 D loss: 0.4927 G loss: 1.996
Epoch: 0001 D loss: 0.4423 G loss: 2.21
Epoch: 0002 D loss: 0.1306 G loss: 3.232
Epoch: 0003 D loss: 0.4884 G loss: 1.736
Epoch: 0004 D loss: 0.4675 G loss: 1.597
Epoch: 0005 D loss: 0.1021 G loss: 3.436
Epoch: 0006 D loss: 0.09623 G loss: 3.703
Epoch: 0007 D loss: 0.2751 G loss: 2.759
Epoch: 0008 D loss: 0.2382 G loss: 2.74
Epoch: 0009 D loss: 0.2016 G loss: 2.879
Epoch: 0010 D loss: 0.2786 G loss: 2.43
Epoch: 0011 D loss: 0.5121 G loss: 2.356
Epoch: 0012 D loss: 0.3614 G loss: 2.576
Epoch: 0013 D loss: 0.4451 G loss: 2.456
Epoch: 0014 D loss: 0.2943 G loss: 2.586
Epoch: 0015 D loss: 0.2807 G loss: 2.709
Epoch: 0016 D loss: 0.3167 G loss: 2.996
Epoch: 0017 D loss: 0.4856 G loss: 2.24
Epoch: 0018 D loss: 0.4196 G loss: 2.365
Epoch: 0019 D loss: 0.416 G loss: 2.708
Epoch: 0020 D loss: 0.3198 G loss: 2.686
Epoch: 0021 D loss: 0.5958 G loss: 2.516
Epoch: 0022 D loss: 0.4401 G loss: 2.614
Epoch: 0023 D loss: 0.3502 G loss: 2.468
Epoch: 0024 D loss: 